In [1]:
import wikipedia
import json
import time
import os

import pandas as pd
import numpy as np
import boto3
import awswrangler as wr

from bs4 import BeautifulSoup
from difflib import SequenceMatcher

# Get the list of IMO numbers from the database

In [2]:
DATABASE = ""
TABLE = ""
OUTPUT_LOCATION = ""

In [3]:
my_session = boto3.session.Session(
    region_name=os.environ['REGION'], 
    aws_access_key_id=os.environ['ACCESS_KEY'], 
    aws_secret_access_key=os.environ['SECRET']
)

In [11]:
query = f"""
    WITH latest_versions AS (
        SELECT CAST(year AS INTEGER) AS year, MAX(CAST(version AS INTEGER)) AS latest_version
        FROM "{DATABASE}"."{TABLE}"
        GROUP BY CAST(year AS INTEGER)
    ),

    latest_data AS (
        SELECT *
        FROM "{DATABASE}"."{TABLE}" se
        JOIN latest_versions lv
        ON CAST(se.year AS INT) = lv.year
        AND CAST(se.version AS INT) = lv.latest_version
    )
    
    SELECT DISTINCT imo_number, name FROM latest_data;
"""

In [12]:
distinc_imo_numbers = wr.athena.read_sql_query(query, database=DATABASE, boto3_session=my_session)

In [6]:
distinc_imo_numbers.head()

,imo_number,name
0,6703343,EQUALITY
1,7225910,AEGEAN ODYSSEY
2,7230599,MARKO POLO
3,7304314,ALBATROS
4,7346221,SUPERFERRY II


In [13]:
distinc_imo_numbers.shape

(24738, 2)

# Filter out the IMO numbers I haven't run through the parser before

In [21]:
with open('../data/raw/ship_particulars/wikipedia_ship_data.json') as f:
    wikipedia_data = json.load(f)

In [22]:
with open('../data/raw/ship_particulars/kaggle_ship_data.json') as f:
    kaggle_data = json.load(f)

In [23]:
with open('../data/raw/ship_particulars/sflcorp_fleet.json') as f:
    sflcorp_data = json.load(f)

In [24]:
with open('../data/raw/ship_particulars/pleiades_fleet.json') as f:
    pleiades_data = json.load(f)

In [25]:
with open('../data/raw/ship_particulars/cmb-tech-fleet.json') as f:
    cmb_tech_data = json.load(f)

In [26]:
ships_acquired = wikipedia_data | kaggle_data | sflcorp_data | pleiades_data | cmb_tech_data

In [27]:
imo_ships_acquired = list(map(int, ships_acquired.keys()))

In [28]:
len(imo_ships_acquired)

4351

In [61]:
imo_numbers_to_query = set(distinc_imo_numbers['imo_number'].to_list()).difference(set(imo_ships_acquired.extend(ships_not_found)))

TypeError: 'NoneType' object is not iterable

In [30]:
len(imo_numbers_to_query)

16774

# Start querying wikipedia for the ships we don't have any information on

In [31]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [32]:
def get_wikipedia_page(page_title: str):
    print('Page title: ', page_title)
    page_object = wikipedia.page(page_title, auto_suggest=False)
    html_page = page_object.html()
    
    return html_page

In [33]:
def get_info_box_from_article(html_page):
    soup = BeautifulSoup(html_page, "html.parser")
    table = soup.find('table', attrs={'class':'infobox'})
    table_body = table.find('tbody')
    
    ship_info_dict = {}
    rows = table_body.find_all('tr', attrs={'style': 'vertical-align:top;'})
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        
        if len(cols) == 2:
            ship_info_dict[cols[0]] = cols[1]
    
    return ship_info_dict


In [67]:
distinc_imo_numbers_subset = distinc_imo_numbers[distinc_imo_numbers['imo_number'].isin(imo_numbers_to_query)].copy()

In [68]:
distinc_imo_numbers_subset.shape

(19043, 2)

In [69]:
ships_not_found = []
ship_info_json = {}
ships_with_info = []
for index, row in distinc_imo_numbers_subset[['imo_number', 'name']].iterrows():
    print(index, ' ', row['imo_number'], ' ',row['name'])
    result = wikipedia.search(f"IMO number: {row['imo_number']}", results = 1)
    
    if not result:
        ships_not_found.append(row['imo_number'])
        time.sleep(10)
        continue
    
    if similar(row['name'].lower(), result[0].lower()) > 0.6:
        print(result)
        time.sleep(5)
        article_html_page = get_wikipedia_page(result[0])
        ship_info = get_info_box_from_article(html_page=article_html_page)
        ship_info_json[row['imo_number']] = ship_info
    print()
    time.sleep(10)

4   7602120   AURELIA

12   7901265   ROSELLA

138   9125372   CELESTINE

147   9129263   BALTIC BRIGHT

148   9131527   EUROCARGO PATRASSO

156   9138393   GRANDE MEDITERRANEO

157   9138422   REPUBBLICA DEL BRASILE

199   9166778   MSC ABY

210   9173068   CLIPPER HARALD

281   9215476   GALLIPOLI SEAWAYS

283   9216298   LNG RIVERS

284   9216303   LNG SOKOTO

310   9226504   TEOMAN A

323   9229910   NORDNES

335   9232890   MSC ENGLAND

371   9243174   MIRADOR EXPRESS

378   9246607   GRANDE AMBURGO

429   9263655   ATHARA

535   9388209   PS HOUSTON

593   9399052   MSC EMANUELA

617   9402471   FULMAR

666   9410753   CMA CGM CALLISTO

711   9421075   MANISA BELLA / LENNEBORG

760   9430272   CELSIUS ROSKILDE

769   9431707   NAVIOS DORADO
770   9431836   HOEGH BEIJING
771   9432048   DEMETRIOS
772   9432139   PARANDOWSKI
774   9432206   BELTNES
775   9432464   DENSA LION
776   9432517   SIDER MONICA
777   9433262   HUDSONGRACHT
778   9433303   LOKHOLMEN
779   9433585   PINK STA

KeyboardInterrupt: 

In [51]:
ship_info_json.keys()

dict_keys([7911545, 9015266, 9056583, 9208629, 9283186, 9283239, 9384992])

In [59]:
ships_not_found.extend(ship_info_json.keys())

In [60]:
len(ships_not_found)

637

In [64]:
imo_ships_acquired.extend(ships_not_found)

In [65]:
imo_numbers_to_query = set(distinc_imo_numbers['imo_number'].to_list()).difference(set(imo_ships_acquired))

In [66]:
len(imo_numbers_to_query)

16137

In [54]:
wikipedia_data = wikipedia_data | ship_info_json

In [55]:
import json
with open('../data/raw/wikipedia_ship_data.json', 'w', encoding='utf-8') as f:
    json.dump(wikipedia_data, f, ensure_ascii=False, indent=4)